In [4]:
import OpenDartReader
import FinanceDataReader as fdr
import pandas as pd
import time
from pathlib import Path

api_key = '85c2367fdad752f376b3947ccdf9998092765f0e'  # OpenDart API KEY

In [3]:
dart = OpenDartReader(api_key)

In [24]:
symbols = fdr.StockListing("KRX")

In [25]:
symbols.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7626 entries, 0 to 7625
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Symbol          7626 non-null   object        
 1   Market          7626 non-null   object        
 2   Name            7626 non-null   object        
 3   Sector          2492 non-null   object        
 4   Industry        2474 non-null   object        
 5   ListingDate     2492 non-null   datetime64[ns]
 6   SettleMonth     2492 non-null   object        
 7   Representative  2492 non-null   object        
 8   HomePage        2341 non-null   object        
 9   Region          2492 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 655.4+ KB


In [26]:
symbols = list(symbols['Symbol'])

In [32]:
symbols

['060310',
 '095570',
 '006840',
 '054620',
 '265520',
 '211270',
 '152100',
 '295820',
 '253150',
 '253160',
 '395750',
 '395760',
 '278420',
 '292750',
 '309210',
 '333940',
 '333950',
 '333960',
 '333970',
 '333980',
 '269530',
 '251590',
 '161510',
 '251600',
 '289670',
 '298340',
 '189400',
 '419650',
 '415920',
 '278620',
 '269540',
 '287180',
 '213630',
 '332610',
 '332620',
 '195970',
 '238670',
 '195980',
 '373530',
 '256450',
 '239660',
 '280920',
 '266550',
 '301400',
 '301410',
 '227830',
 '122090',
 '328370',
 '301440',
 '376250',
 '027410',
 '282330',
 '032790',
 '138930',
 '001460',
 '001465',
 '013720',
 '001040',
 '079160',
 '035760',
 '311690',
 '00104K',
 '000120',
 '011150',
 '011155',
 '001045',
 '097950',
 '097955',
 '051500',
 '058820',
 '023460',
 '056730',
 '056000',
 '065770',
 '083660',
 '000590',
 '012030',
 '404950',
 '367340',
 '367360',
 '016610',
 '005830',
 '000990',
 '000995',
 '139130',
 '001530',
 '000210',
 '001880',
 '000215',
 '375500',
 '37550K',

In [29]:
stock_names = ['삼성전자', 'LG전자', '현대자동차']

In [30]:
stock_names

['삼성전자', 'LG전자', '현대자동차']

In [33]:
# 데이터 초기화. df2 는 정리된 데이터를 모으는 곳이며 얻고자 하는 데이터 이름을
# column 화 시킴.  빈 index 를 넣으면 추가가 안되므로 임으로 하나 넣자.
df2 = pd.DataFrame(columns=['종목코드', '유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                            '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01'])

# '11013' = 1분기보고서, '11012' = 반기보고서, '11014' = 3분기보고서, '11011' = 사업보고서
reprt_code = ['11013', '11012', '11014', '11011']

In [34]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 1900-01-01 to 1900-01-01
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   종목코드      0 non-null      object
 1   유동자산      0 non-null      object
 2   부채총계      0 non-null      object
 3   자본총계      0 non-null      object
 4   매출액       0 non-null      object
 5   매출총이익     0 non-null      object
 6   영업이익      0 non-null      object
 7   당기순이익     0 non-null      object
 8   영업활동현금흐름  0 non-null      object
 9   잉여현금흐름    0 non-null      object
dtypes: object(10)
memory usage: 88.0+ bytes


In [38]:
# 데이터를 얻기위한 반복문 시작
for stock_name in stock_names:
    # 정리 완료된 파일 저장하기 위한 경로 및 이름. result_종목이름.xlsx 형태로 저장된다.
    filePath = 'C:/workspace/capstone-finance/finance-data/data/'
    Path(filePath).mkdir(exist_ok=True)
    fileName = f'result_{str(stock_name)}.csv'
    for i in range(2015, 2022):  # OpenDart는 2015년부터 정보를 제공한다.

        # 더미 리스트 초기화. 1 ~ 4 분기 데이터를 합할 예정이므로 4 크기 만큼의 리스트 선언.
        current_assets = [0, 0, 0, 0]  # 유동자산
        liabilities = [0, 0, 0, 0]  # 부채총계
        equity = [0, 0, 0, 0]  # 자본총계
        revenue = [0, 0, 0, 0]  # 매출액
        gross_marin = [0, 0, 0, 0]  # 매출총이익 #grossMargin
        EBIT = [0, 0, 0, 0]  # 영업이익 #EBIT
        net_income = [0, 0, 0, 0]  # 당기순이익
        cfo = [0, 0, 0, 0]  # 영업활동현금흐름
        cfi = [0, 0, 0, 0]  # 투자활동현금흐름
        fcf = [0, 0, 0, 0]  # 잉여현금흐름 : 편의상 영업활동 - 투자활동 현금흐름으로 계산

        for j, k in enumerate(reprt_code):
            df1 = pd.DataFrame()  # Raw Data
            if str(type(dart.finstate_all('004840', i, reprt_code=k, fs_div='CFS'))) == "<class 'NoneType'>":
                pass

            # 타입이 NoneType 이 아니면 읽어온다.
            else:
                df1 = df1.append(dart.finstate_all(stock_name, i, reprt_code=k, fs_div='CFS'))
                # 재무상태표 부분
                condition = (df1.sj_nm == '재무상태표') & (df1.account_nm == '유동자산')  # 유동자산
                condition_2 = (df1.sj_nm == '재무상태표') & (df1.account_nm == '부채총계')  # 부채총계
                condition_3 = (df1.sj_nm == '재무상태표') & ((df1.account_nm == '자본총계') | (df1.account_nm == '반기말자본') | (
                        df1.account_nm == '3분기말자본') | (df1.account_nm == '분기말자본') | (
                                                                df1.account_nm == '1분기말자본'))  #자본총계
                # 손익계산서 부분
                condition_4 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '매출액') | (df1.account_nm == '수익(매출액)'))
                condition_5 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (df1.account_nm == '매출총이익')
                condition_6 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '영업이익(손실)') | (df1.account_nm == '영업이익'))
                condition_7 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '당기순이익(손실)') | (df1.account_nm == '당기순이익') |
                        (df1.account_nm == '분기순이익') | (df1.account_nm == '분기순이익(손실)') | (
                                df1.account_nm == '반기순이익') | (df1.account_nm == '반기순이익(손실)') |
                        (df1.account_nm == '연결분기순이익') | (df1.account_nm == '연결반기순이익') | (
                                df1.account_nm == '연결당기순이익') | (df1.account_nm == '연결분기(당기)순이익') | (
                                df1.account_nm == '연결반기(당기)순이익') |
                        (df1.account_nm == '연결분기순이익(손실)'))
                # 현금흐름표 부분
                condition_8 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '영업활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))
                condition_9 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '투자활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))

                stock_code = stock_name
                current_assets[j] = int(df1.loc[condition].iloc[0]['thstrm_amount'])
                liabilities[j] = int(df1.loc[condition_2].iloc[0]['thstrm_amount'])
                equity[j] = int(df1.loc[condition_3].iloc[0]['thstrm_amount'])
                revenue[j] = int(df1.loc[condition_4].iloc[0]['thstrm_amount'])
                gross_marin[j] = int(df1.loc[condition_5].iloc[0]['thstrm_amount'])
                EBIT[j] = int(df1.loc[condition_6].iloc[0]['thstrm_amount'])
                net_income[j] = int(df1.loc[condition_7].iloc[0]['thstrm_amount'])
                cfo[j] = int(df1.loc[condition_8].iloc[0]['thstrm_amount'])
                cfi[j] = int(df1.loc[condition_9].iloc[0]['thstrm_amount'])
                fcf[j] = (cfo[j] - cfi[j])

                if k == '11013':  # 1분기.
                    path_string = str(i) + '-03-31'
                elif k == '11012':  # 2분기
                    path_string = str(i) + '-06-30'
                elif k == '11014':  # 3분기
                    path_string = str(i) + '-09-30'
                else:  # 4분기. 1 ~ 3분기 데이터를 더한다음 사업보고서에서 빼야 함
                    path_string = str(i) + '-12-30'
                    revenue[j] = revenue[j] - (revenue[0] + revenue[1] + revenue[2])
                    gross_marin[j] = gross_marin[j] - (gross_marin[0] + gross_marin[1] + gross_marin[2])
                    EBIT[j] = EBIT[j] - (EBIT[0] + EBIT[1] + EBIT[2])
                    net_income[j] = net_income[j] - (net_income[0] + net_income[1] + net_income[2])
                    fcf[j] = fcf[j] - (fcf[0] + fcf[1] + fcf[2])

                # 데이터프레임에 저장하는 부분
                df2.loc[path_string] = [stock_code, current_assets[j], liabilities[j], equity[j],
                                        revenue[j], gross_marin[j], EBIT[j], net_income[j], cfo[j], fcf[j]]
                df2.tail()  # 데이터프레임 끝에 저장한다.
            # 너무 잦은 요청이 있을 경우 OpenDart API 측에서 IP 를 차단하니 텀을 둔다.
            time.sleep(0.1)

    # 원본 dataframe 에도 영향을 끼치게끔 (inplace=True) 첫 행 drop.
    # df2.drop(['1900-01-01'], inplace=True) # 첫 행 drop

    # 파일 저장. fileName 에 명시된 경로에 각 종목코드별로 다른 이름으로 저장
    df2.to_csv(filePath + fileName)

    # 그 다음 종목의 데이터를 얻기 위해 df2 재초기화
    df2 = pd.DataFrame(columns=['종목코드', '유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익', '당기순이익', '영업활동현금흐름', '잉여현금흐름'],
                       index=['1900-01-01'])  # 정리된 Data. 빈 index 를 넣으면 추가가 안되므로 하나 넣자.

In [1]:
# 데이터를 얻기위한 반복문 시작
for symbol in symbols:
    # 정리 완료된 파일 저장하기 위한 경로 및 이름. result_종목이름.xlsx 형태로 저장된다.
    filePath = 'C:/workspace/capstone-finance/finance-data/data/'
    Path(filePath).mkdir(exist_ok=True)
    fileName = f'2022_1_KRX.csv'
    for i in range(2015, 2022):  # OpenDart는 2015년부터 정보를 제공한다.

        # 더미 리스트 초기화. 1 ~ 4 분기 데이터를 합할 예정이므로 4 크기 만큼의 리스트 선언.
        current_assets = [0, 0, 0, 0]  # 유동자산
        liabilities = [0, 0, 0, 0]  # 부채총계
        equity = [0, 0, 0, 0]  # 자본총계
        revenue = [0, 0, 0, 0]  # 매출액
        gross_marin = [0, 0, 0, 0]  # 매출총이익 #grossMargin
        EBIT = [0, 0, 0, 0]  # 영업이익 #EBIT
        net_income = [0, 0, 0, 0]  # 당기순이익
        cfo = [0, 0, 0, 0]  # 영업활동현금흐름
        cfi = [0, 0, 0, 0]  # 투자활동현금흐름
        fcf = [0, 0, 0, 0]  # 잉여현금흐름 : 편의상 영업활동 - 투자활동 현금흐름으로 계산

        for j, k in enumerate(reprt_code):
            df1 = pd.DataFrame()  # Raw Data
            if str(type(dart.finstate_all('004840', i, reprt_code=k, fs_div='CFS'))) == "<class 'NoneType'>":
                pass

            # 타입이 NoneType 이 아니면 읽어온다.
            else:
                df1 = df1.append(dart.finstate_all(symbol, i, reprt_code=k, fs_div='CFS'))
                # 재무상태표 부분
                condition = (df1.sj_nm == '재무상태표') & (df1.account_nm == '유동자산')  # 유동자산
                condition_2 = (df1.sj_nm == '재무상태표') & (df1.account_nm == '부채총계')  # 부채총계
                condition_3 = (df1.sj_nm == '재무상태표') & ((df1.account_nm == '자본총계') | (df1.account_nm == '반기말자본') | (
                        df1.account_nm == '3분기말자본') | (df1.account_nm == '분기말자본') | (
                                                                df1.account_nm == '1분기말자본'))  #자본총계
                # 손익계산서 부분
                condition_4 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '매출액') | (df1.account_nm == '수익(매출액)'))
                condition_5 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (df1.account_nm == '매출총이익')
                condition_6 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '영업이익(손실)') | (df1.account_nm == '영업이익'))
                condition_7 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '당기순이익(손실)') | (df1.account_nm == '당기순이익') |
                        (df1.account_nm == '분기순이익') | (df1.account_nm == '분기순이익(손실)') | (
                                df1.account_nm == '반기순이익') | (df1.account_nm == '반기순이익(손실)') |
                        (df1.account_nm == '연결분기순이익') | (df1.account_nm == '연결반기순이익') | (
                                df1.account_nm == '연결당기순이익') | (df1.account_nm == '연결분기(당기)순이익') | (
                                df1.account_nm == '연결반기(당기)순이익') |
                        (df1.account_nm == '연결분기순이익(손실)'))
                # 현금흐름표 부분
                condition_8 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '영업활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))
                condition_9 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '투자활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))

                stock_code = symbol
                current_assets[j] = int(df1.loc[condition].iloc[0]['thstrm_amount'])
                liabilities[j] = int(df1.loc[condition_2].iloc[0]['thstrm_amount'])
                equity[j] = int(df1.loc[condition_3].iloc[0]['thstrm_amount'])
                revenue[j] = int(df1.loc[condition_4].iloc[0]['thstrm_amount'])
                gross_marin[j] = int(df1.loc[condition_5].iloc[0]['thstrm_amount'])
                EBIT[j] = int(df1.loc[condition_6].iloc[0]['thstrm_amount'])
                net_income[j] = int(df1.loc[condition_7].iloc[0]['thstrm_amount'])
                cfo[j] = 0
                cfi[j] = 0
                if df1.loc[condition_8] & df1.loc[condition_8].iloc[0]:
                    cfo[j] = int(df1.loc[condition_8].iloc[0]['thstrm_amount'])
                if df1.loc[condition_9] & df1.loc[condition_9].iloc[0]:
                    cfi[j] = int(df1.loc[condition_9].iloc[0]['thstrm_amount'])
                fcf[j] = (cfo[j] - cfi[j])

                if k == '11013':  # 1분기.
                    path_string = str(i) + '-03-31'
                elif k == '11012':  # 2분기
                    path_string = str(i) + '-06-30'
                elif k == '11014':  # 3분기
                    path_string = str(i) + '-09-30'
                else:  # 4분기. 1 ~ 3분기 데이터를 더한다음 사업보고서에서 빼야 함
                    path_string = str(i) + '-12-30'
                    revenue[j] = revenue[j] - (revenue[0] + revenue[1] + revenue[2])
                    gross_marin[j] = gross_marin[j] - (gross_marin[0] + gross_marin[1] + gross_marin[2])
                    EBIT[j] = EBIT[j] - (EBIT[0] + EBIT[1] + EBIT[2])
                    net_income[j] = net_income[j] - (net_income[0] + net_income[1] + net_income[2])
                    fcf[j] = fcf[j] - (fcf[0] + fcf[1] + fcf[2])

                # 데이터프레임에 저장하는 부분
                df2.loc[path_string] = [stock_code, current_assets[j], liabilities[j], equity[j],
                                        revenue[j], gross_marin[j], EBIT[j], net_income[j], cfo[j], fcf[j]]
                df2.tail()  # 데이터프레임 끝에 저장한다.
            # 너무 잦은 요청이 있을 경우 OpenDart API 측에서 IP 를 차단하니 텀을 둔다.
            time.sleep(0.1)

    # 원본 dataframe 에도 영향을 끼치게끔 (inplace=True) 첫 행 drop.
    # df2.drop(['1900-01-01'], inplace=True) # 첫 행 drop

    # 파일 저장. fileName 에 명시된 경로에 각 종목코드별로 다른 이름으로 저장
    df2.to_csv(filePath + fileName)

    # 그 다음 종목의 데이터를 얻기 위해 df2 재초기화
    df2 = pd.DataFrame(columns=['종목코드', '유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익', '당기순이익', '영업활동현금흐름', '잉여현금흐름'],
                       index=['1900-01-01'])  # 정리된 Data. 빈 index 를 넣으면 추가가 안되므로 하나 넣자.

NameError: name 'symbols' is not defined